# Read the data

In [12]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
data=pd.read_csv("complete.csv",on_bad_lines='skip')
data.head(10)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
5,10/10/1961 19:00,bristol,tn,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.5950000,-82.188889
6,10/10/1965 21:00,penarth (uk/wales),NaN,gb,circle,180,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000
7,10/10/1965 23:45,norwalk,ct,us,disk,1200,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.1175000,-73.408333
8,10/10/1966 20:00,pell city,al,us,disk,180,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.5861111,-86.286111
9,10/10/1966 21:00,live oak,fl,us,disk,120,several minutes,Saucer zaps energy from powerline as my pregna...,5/11/2005,30.2947222,-82.984167


# Data Cleaning
1. Drop NaN Country
2. Get the Posted Year
3. Get US data Only

In [13]:
data=data.dropna(subset=['country'])
data["country"].unique()
data["Year_Posted"]=data["date posted"].str[-4:]
clean_data=data[["city","state","country","shape","duration (seconds)","latitude","longitude","Year_Posted"]]
us_data=clean_data[clean_data["country"]=="us"]

us_data["duration"]=us_data["duration (seconds)"].str.extract('(^\d*)')
#us_data["lon"]=us_data["longitude"].str.extract('(^\d*)')
us_data=us_data.drop(["duration (seconds)"], axis=1)
us_data.head(10)

,city,state,country,shape,latitude,longitude,Year_Posted,duration
0,san marcos,tx,us,cylinder,29.8830556,-97.941111,2004,2700
3,edna,tx,us,circle,28.9783333,-96.645833,2004,20
4,kaneohe,hi,us,light,21.4180556,-157.803611,2004,900
5,bristol,tn,us,sphere,36.5950000,-82.188889,2007,300
7,norwalk,ct,us,disk,41.1175000,-73.408333,1999,1200
8,pell city,al,us,disk,33.5861111,-86.286111,2009,180
9,live oak,fl,us,disk,30.2947222,-82.984167,2005,120
10,hawthorne,ca,us,circle,33.9163889,-118.351667,2003,300
11,brevard,nc,us,fireball,35.2333333,-82.734444,2008,180
12,bellmore,ny,us,disk,40.6686111,-73.527500,2000,1800


# Matedata Summary
1. Year obeservations based on State
2. Year obeservations based on City
3. Year obeservations based on Shape

In [14]:
yearly_by_shape=us_data.groupby(["Year_Posted","shape"])["city"].count()
yearly_by_shape=yearly_by_shape.reset_index()
max_shape=yearly_by_shape.sort_values('city', ascending=False).drop_duplicates(['Year_Posted'])

In [15]:
yearly_by_state=us_data.groupby(["Year_Posted","state"])["city"].count()
yearly_by_state=yearly_by_state.reset_index()
yearly_by_state["state"]=yearly_by_state["state"].str.upper()
yearly_by_state
max_state=yearly_by_state.sort_values('city', ascending=False).drop_duplicates(['Year_Posted'])

In [16]:
yearly_by_city=us_data.groupby(["Year_Posted","city"])["state"].count()
yearly_by_city=yearly_by_city.reset_index()
max_city=yearly_by_city.sort_values('state', ascending=False).drop_duplicates(['Year_Posted'])

In [17]:
json_object={}
json_object["names"]=list(us_data["Year_Posted"].unique())

In [21]:
max_shape.head(3)

,Year_Posted,shape,city
318,2012,light,1535
339,2013,light,1441
255,2009,light,1219


In [22]:
max_state.head(3)

,Year_Posted,state,city
712,2012,CA,805
560,2009,CA,778
763,2013,CA,693


In [23]:
max_city.head(3)

,Year_Posted,city,state
2378,1999,seattle,91
12238,2004,tinley park,84
31446,2012,seattle,47


In [42]:
city_temp=max_city[max_city["Year_Posted"]==json_object["names"][0]]
city_temp.iloc[0]['city']




'tinley park'

In [43]:
mate_list=[]
for i in json_object["names"]:
    
    results={}
    results["id"]=i
    city_temp=max_city[max_city["Year_Posted"]==i]
    results[city_temp.iloc[0]['city']]=city_temp.iloc[0]['state']
    state_temp=max_state[max_state["Year_Posted"]==i]
    results[state_temp.iloc[0]['state']]=city_temp.iloc[0]['city']
    shape_temp=max_shape[max_shape["Year_Posted"]==i]
    results[shape_temp.iloc[0]['shape']]=city_temp.iloc[0]['city'] 
    
    mate_list.append(results)

In [44]:
mate_list

[{'id': '2004', 'tinley park': 84},
 {'id': '2007', 'seattle': 30},
 {'id': '1999', 'seattle': 91},
 {'id': '2009', 'san diego': 38},
 {'id': '2005', 'phoenix': 46},
 {'id': '2003', 'san diego': 34},
 {'id': '2008', 'phoenix': 34},
 {'id': '2000', 'gilbert': 36},
 {'id': '2010', 'seattle': 30},
 {'id': '2014', 'albuquerque': 19},
 {'id': '2001', 'rockford': 41},
 {'id': '2002', 'seattle': 40},
 {'id': '2012', 'seattle': 47},
 {'id': '2006', 'phoenix': 36},
 {'id': '2013', 'portland': 39},
 {'id': '1998', 'portland': 8},
 {'id': '2011', 'seattle': 29}]

Create Sample Data for Each State

In [10]:
yearly_by_state=us_data.groupby(["Year_Posted","state"])["city"].count()
yearly_by_state=yearly_by_state.reset_index()
yearly_by_state["state"]=yearly_by_state["state"].str.upper()
yearly_by_state

,Year_Posted,state,city
0,1998,AK,8
1,1998,AL,5
2,1998,AR,2
3,1998,AZ,41
4,1998,CA,114
...,...,...,...
854,2014,VT,13
855,2014,WA,133
856,2014,WI,44
857,2014,WV,25


In [11]:
sample=[]
for i in list(yearly_by_state["Year_Posted"].unique()):
    results={}
    results["id"]=i
    selected_year_data=yearly_by_state[yearly_by_state["Year_Posted"]==i]
    results["state_name"]=list(selected_year_data["state"])
    results["count"]=list(selected_year_data["city"])
    sample.append(results)

In [12]:
shape_metadata["sample"]=sample

Create sample data based on shape

In [15]:
yearly_by_shape=us_data.groupby(["Year_Posted","shape"])["city"].count()
yearly_by_shape=yearly_by_shape.reset_index()

In [16]:
yearly_by_shape

,Year_Posted,shape,city
0,1998,changing,14
1,1998,chevron,6
2,1998,cigar,14
3,1998,circle,40
4,1998,cone,4
...,...,...,...
363,2014,rectangle,39
364,2014,sphere,232
365,2014,teardrop,15
366,2014,triangle,260


In [18]:
shape_sample=[]
for i in list(yearly_by_shape["Year_Posted"].unique()):
    results={}
    results["id"]=i
    selected_year_data=yearly_by_shape[yearly_by_shape["Year_Posted"]==i]
    results["state_name"]=list(selected_year_data["shape"])
    results["count"]=list(selected_year_data["city"])
    shape_sample.append(results)

In [19]:
shape_metadata["shape_sample"]=shape_sample

In [20]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    
# Serializing json
json_object = json.dumps(shape_metadata, cls=NpEncoder)
 
# Writing to sample.json
with open("shape_metadata.json", "w") as outfile:
    outfile.write(json_object)

sample

In [176]:
yearly_by_State=us_data.groupby(["Year_Posted","state"])["city"].count()
yearly_by_State=yearly_by_State.reset_index()
yearly_by_State["state"]=yearly_by_State["state"].str.upper()
yearly_by_State

,Year_Posted,state,city
0,1998,AK,8
1,1998,AL,5
2,1998,AR,2
3,1998,AZ,41
4,1998,CA,114
...,...,...,...
854,2014,VT,13
855,2014,WA,133
856,2014,WI,44
857,2014,WV,25


In [6]:
new_data={}
new_data["names"]=clean_data["Year_Posted"].unique()
new_data["names"]

array(['2004', '2008', '2007', '2006', '1999', '2009', '2005', '2003',
       '2000', '2010', '2014', '2001', '2002', '2012', '2013', '1998',
       '2011'], dtype=object)

Matadata

In [8]:
clean_data.groupby(["Year_Posted",""]).count()

Year_Posted  country
1998         au            8
             gb            4
             us          801
1999         au           21
             ca            7
                        ... 
2014         au           20
             ca           76
             de            4
             gb           31
             us         2815
Name: country, Length: 83, dtype: int64

In [9]:
clean_data.groupby(["Year_Posted","shape"])["country"].count()

Year_Posted  shape    
1998         changing      14
             chevron        6
             cigar         14
             circle        41
             cone           4
                         ... 
2014         rectangle     40
             sphere       235
             teardrop      16
             triangle     276
             unknown      205
Name: country, Length: 368, dtype: int64

In [70]:


    

import json

# Serializing json
json_object = json.dumps(new_data, indent=4)
 
# Writing to sample.json
with open("complete.json", "w") as outfile:
    outfile.write(json_object)